# PingPong Latency Matrix

This shows results of running a pingpong latency test on all nodes in ReFrame partition(s).

The result matrices show latency against MPI rank pairs. The rank is not linked to specific hostnames here but as the test uses one rank per node ranks should generally follow the order of nodes shown by e.g. Slurm's `sinfo`.

The final section shows summmary statistics for all partitions.

- TODO: move code into modules

In [46]:
import sys, statistics
from pprint import pprint
sys.path.extend(('../../reframe', '../../')) # the `modules` package imports `reframe` so need that in sys.modules
import modules
import pandas as pd
import matplotlib as mpl
from IPython.display import display, Markdown

In [54]:
# find outputs:
paths = [p for p in modules.utils.find_run_outputs(root='../../output', test='Nxnlatbw') if 'build' not in p]
meta = [modules.utils.parse_path_metadata(p) for p in paths]

# load and plot:
summary = {} # sys:part (str) -> stats (dict)
for ix, path in enumerate(paths):
    syspart = '%s:%s' % (meta[ix]['sysname'], meta[ix]['partition'])
    display(Markdown('# %s' % syspart))
    
    # read latencies:
    latencies = {}
    with open(path) as nxn_f:
        for ln, line in enumerate(nxn_f):
            vals = line.strip().split(',')
            if vals[0] == 'src':
                continue
            if len(vals) != 4:
                print('warning: skipping line %i (%i values) beginning: %r ...' % (ln, len(vals), line[0:50]))
                continue
            rankA, rankB, lat, bw = int(vals[0]), int(vals[1]), float(vals[2]), float(vals[3])
            latencies[rankA, rankB] = lat
    
    # get list of node IDs:
    rankAs = sorted(set(k[0] for k in latencies))
    rankBs = sorted(set(k[1] for k in latencies))
    # TODO: could crosscheck these!
    
    # create a dataframe using a list-of-lists:
    lat_list = []
    for rankA in rankAs: # row
        row = [latencies.get((rankA, rankB)) for rankB in rankBs]
        lat_list.append(row)
    df = pd.DataFrame(lat_list)
    dfv = df.stack() # flatten
    summary[syspart] = {
        'count': len(dfv),
        'min': dfv.min(),
        'max': dfv.max(),
        'mean': dfv.mean(),
        'stdev': dfv.std(),
    }
    sdf = df.style.background_gradient(cmap=mpl.pyplot.get_cmap('Reds')).set_precision(2)
    display(sdf)

# tabulate summary
display(Markdown('# Summary'))
display(pd.DataFrame(summary).sort_index(axis=1)) # sort columns by name
    


# csd3:cclake-ib-gcc9-ompi3-ucx

/home/hpcbras1/miniconda3/envs/hpc-tests/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


# csd3:cclake-ib-icc19-impi19-ucx

# csd3:cclake-roce-gcc9-ompi3-ucx

# csd3:cclake-roce-icc19-impi19-ucx

...

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,nan,1.36,1.36,2.06,2.04,1.37,2.06,1.38,2.05,1.36,2.05,2.06,1.36,1.36,1.36,2.04
1,1.37,nan,1.36,2.05,2.05,1.36,2.05,1.38,2.05,1.37,2.04,2.05,1.37,1.37,1.37,2.04
2,1.36,1.37,nan,2.05,2.05,1.37,2.05,1.38,2.04,1.36,2.04,2.04,1.36,1.37,1.37,2.04
3,2.04,2.05,2.04,nan,1.37,2.06,1.37,2.06,1.37,2.05,1.36,1.36,2.04,2.06,2.05,1.36
4,2.04,2.06,2.05,1.37,nan,2.05,1.37,2.07,1.37,2.05,1.37,1.37,2.05,2.05,2.05,1.38
5,1.36,1.37,1.36,2.05,2.06,nan,2.05,1.38,2.05,1.38,2.06,2.06,1.36,1.37,1.36,2.05
6,2.05,2.05,2.05,1.37,1.37,2.04,nan,2.05,1.37,2.06,1.36,1.37,2.05,2.04,2.04,1.37
7,1.37,1.37,1.37,2.06,2.06,1.37,2.04,nan,2.06,1.38,2.05,2.05,1.37,1.38,1.37,2.05
8,2.04,2.05,2.04,1.37,1.36,2.05,1.36,2.07,nan,2.13,1.36,1.38,2.05,2.06,2.06,1.36
9,1.44,1.40,1.36,2.05,2.05,1.37,2.06,1.38,2.05,nan,2.05,2.06,1.37,1.39,1.38,2.04


# alaska:ib-gcc9-openmpi4-ucx

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,nan,0.82,0.82,0.96,0.97,0.87,0.96,0.81,0.96,0.86,0.96,0.96,0.82,0.82,0.81,0.95
1,0.83,nan,0.82,0.97,0.97,0.87,0.97,0.82,0.96,0.87,0.97,0.96,0.82,0.83,0.82,0.96
2,0.83,0.82,nan,0.96,0.97,0.87,0.96,0.81,0.96,0.87,0.96,0.96,0.82,0.83,0.82,0.95
3,0.96,0.96,0.96,nan,0.82,1.02,0.81,0.95,0.82,1.01,0.81,0.81,0.96,0.96,0.96,0.81
4,0.97,0.97,0.97,0.82,nan,1.01,0.81,0.96,0.82,1.01,0.82,0.81,0.96,0.96,0.96,0.81
5,0.86,0.87,0.87,1.01,1.01,nan,1.01,0.87,1.01,0.91,1.01,1.01,0.87,0.86,0.87,1.01
6,0.95,0.97,0.96,0.80,0.80,1.01,nan,0.96,0.82,1.00,0.81,0.81,0.96,0.96,0.95,0.80
7,0.81,0.84,0.85,0.96,0.96,0.88,0.96,nan,0.97,0.85,0.96,0.96,0.82,0.81,0.81,0.95
8,0.96,0.96,0.96,0.81,0.81,1.01,0.81,0.96,nan,1.01,0.82,0.82,0.96,0.96,0.97,0.82
9,0.86,0.87,0.86,1.02,1.02,0.92,1.01,0.86,1.02,nan,1.02,1.02,0.87,0.86,0.87,1.01


# Summary

,alaska:ib-gcc9-openmpi4-ucx,alaska:roce-gcc9-openmpi4-ucx,csd3:cclake-ib-gcc9-ompi3-ucx,csd3:cclake-ib-icc19-impi19-ucx,csd3:cclake-roce-gcc9-ompi3-ucx,csd3:cclake-roce-icc19-impi19-ucx
count,240.000000,240.000000,3080.000000,3080.000000,3080.000000,3080.000000
min,0.800000,1.350000,0.960000,1.010000,1.390000,1.430000
max,1.020000,2.140000,1.160000,1.120000,1.610000,1.600000
mean,0.904417,1.728792,1.001347,1.019088,1.444273,1.468510
stdev,0.075160,0.341752,0.036940,0.007737,0.034141,0.021099
